# RAG 파이프라인 테스트 및 성능 측정

백엔드 RAG 시스템의 7단계 파이프라인 테스트

## 1. 환경 설정

In [33]:
import sys
import os
import time
import asyncio
import pandas as pd
from datetime import datetime

# 환경 변수 설정 (.env 파일 로드)
env_path = '/Users/kimjm/Desktop/3rd-proj/3rd-proj/back-end/zip_fit/.env'
with open(env_path, 'r') as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#') and '=' in line:
            key, value = line.split('=', 1)
            os.environ[key.strip()] = value.strip()

# 백엔드 모듈 임포트
backend_dir = '/Users/kimjm/Desktop/3rd-proj/3rd-proj/back-end/zip_fit'
sys.path.insert(0, backend_dir)
import llm_handler
import gongo
import config

print(f"모델: {config.EMBEDDING_MODEL_NAME}")
print(f"Reranker: {config.USE_RERANKER}")
print(f"LLM: {config.OPENAI_MODEL}")

모델: BAAI/bge-m3
Reranker: True
LLM: gpt-4o


In [34]:
# 백엔드 모듈 재임포트 (코드 변경 후 실행)
import importlib
importlib.reload(llm_handler)
print("✅ llm_handler 모듈 재로드 완료")

✅ llm_handler 모듈 재로드 완료


## 2. 테스트 케이스 정의

In [ ]:
test_queries = [
    {"id": "TC-01", "query": "수원시 행복주택 공고 알려줘", "type": "멀티쿼리+하이브리드검색"},
]

for tc in test_queries:
    print(f"{tc['id']}: {tc['query']} [{tc['type']}]")

TC-01: 수원시 행복주택 공고 알려줘 [멀티쿼리+하이브리드검색]
TC-02: 수원매산 A1블록 행복주택 신청자격 알려줘 [신청자격]
TC-03: 수원매산 A1블록 행복주택 우선공급 선정기준 알려줘 [우선공급 선정기준]
TC-04: 수원매산 A1블록 행복주택 어느 지역 사람이 신청할 수 있어? [지역 요건]


## 3. 전체 파이프라인 테스트

In [36]:
async def test_pipeline(query: str):
    print(f"\n질문: {query}")
    print("=" * 60)
    
    start = time.time()
    timings = {}
    
    # 1. 질문 재구성
    t = time.time()
    query_analysis = await llm_handler.rewrite_query(query, [])
    timings['재구성'] = time.time() - t
    print(f"[1] 질문 재구성: {timings['재구성']:.2f}s")
    print(f"    -> {query_analysis.get('rewritten_question')}")
    
    # 2. 멀티쿼리
    t = time.time()
    multi_queries = await llm_handler.generate_multi_queries(query, query_analysis, num_queries=1)
    timings['멀티쿼리'] = time.time() - t
    print(f"[2] 멀티쿼리: {timings['멀티쿼리']:.2f}s ({len(multi_queries)}개)")
    
    # 3. 하이브리드 검색
    t = time.time()
    search_results = await gongo.multi_query_hybrid_search(query_analysis, multi_queries)
    timings['검색'] = time.time() - t
    print(f"[3] 하이브리드 검색: {timings['검색']:.2f}s ({len(search_results)}개)")
    
    if not search_results:
        return {
            "timings": timings, 
            "total": time.time() - start, 
            "error": "no_results",
            "query_analysis": query_analysis,
            "multi_queries": multi_queries
        }
    
    # 4. 리랭킹
    t = time.time()
    reranked = await gongo.rerank_results(query_analysis.get('rewritten_question', query), search_results, top_k=25)
    timings['리랭킹'] = time.time() - t
    print(f"[4] 리랭킹: {timings['리랭킹']:.2f}s (top_k=25)")
    
    # 5. 청크 병합
    t = time.time()
    merged = await gongo.merge_chunks(reranked)
    timings['병합'] = time.time() - t
    print(f"[5] 청크 병합: {timings['병합']:.2f}s ({len(merged)}개 공고)")
    
    # 6. 컨텍스트 구성
    t = time.time()
    context = gongo.build_context(merged)
    timings['컨텍스트'] = time.time() - t
    print(f"[6] 컨텍스트: {timings['컨텍스트']:.2f}s ({len(context)}자)")
    
    # 7. 답변 생성
    t = time.time()
    answer = await llm_handler.generate_answer(query_analysis.get('rewritten_question', query), context, [])
    timings['답변'] = time.time() - t
    print(f"[7] 답변 생성: {timings['답변']:.2f}s ({len(answer)}자)")
    
    total = time.time() - start
    print(f"\n총 소요 시간: {total:.2f}s")
    
    return {
        "timings": timings,
        "total": total,
        "answer": answer,
        "search_count": len(search_results),
        "merged_count": len(merged),
        "query_analysis": query_analysis,
        "multi_queries": multi_queries,
        "search_results": search_results[:5],  # 상위 5개만 저장
        "reranked": reranked[:5],  # 상위 5개만 저장
        "merged": merged[:3],  # 상위 3개만 저장
        "context": context
    }

# 모든 테스트 실행
all_results = []
for tc in test_queries:
    result = await test_pipeline(tc['query'])
    result['tc_id'] = tc['id']
    result['query'] = tc['query']
    result['type'] = tc['type']
    all_results.append(result)
    await asyncio.sleep(1)


질문: 수원시 행복주택 공고 알려줘
[1] 질문 재구성: 1.85s
    -> 수원시 행복주택 공고 알려줘
[2] 멀티쿼리: 1.13s (2개)
[3] 하이브리드 검색: 2.03s (19개)
[4] 리랭킹: 13.22s (top_k=25)
[5] 청크 병합: 0.00s (9개 공고)
[6] 컨텍스트: 0.00s (13907자)
[7] 답변 생성: 9.63s (1906자)

총 소요 시간: 27.86s

질문: 수원매산 A1블록 행복주택 신청자격 알려줘
[1] 질문 재구성: 2.16s
    -> 수원매산 A1블록 행복주택 신청자격 알려줘
[2] 멀티쿼리: 2.00s (2개)
[3] 하이브리드 검색: 1.16s (17개)
[4] 리랭킹: 10.48s (top_k=25)
[5] 청크 병합: 0.00s (10개 공고)
[6] 컨텍스트: 0.00s (14734자)
[7] 답변 생성: 4.59s (501자)

총 소요 시간: 20.39s

질문: 수원매산 A1블록 행복주택 우선공급 선정기준 알려줘
[1] 질문 재구성: 1.58s
    -> 수원매산 A1블록 행복주택 우선공급 선정기준 알려줘
[2] 멀티쿼리: 1.41s (2개)
[3] 하이브리드 검색: 1.01s (19개)
[4] 리랭킹: 11.61s (top_k=25)
[5] 청크 병합: 0.00s (9개 공고)
[6] 컨텍스트: 0.00s (16855자)
[7] 답변 생성: 3.55s (286자)

총 소요 시간: 19.17s

질문: 수원매산 A1블록 행복주택 어느 지역 사람이 신청할 수 있어?
[1] 질문 재구성: 1.15s
    -> 수원매산 A1블록 행복주택 어느 지역 사람이 신청할 수 있어?
[2] 멀티쿼리: 1.53s (2개)
[3] 하이브리드 검색: 1.19s (15개)
[4] 리랭킹: 8.70s (top_k=25)
[5] 청크 병합: 0.00s (7개 공고)
[6] 컨텍스트: 0.00s (10805자)
[7] 답변 생성: 2.27s (380자)

총 소요 시간: 14.86s


## 4. 결과 분석

In [37]:
# DataFrame 생성
data = []
for r in all_results:
    if 'error' not in r:
        data.append({
            'TC': r['tc_id'],
            '질문': r['query'],
            '재구성': r['timings']['재구성'],
            '멀티쿼리': r['timings']['멀티쿼리'],
            '검색': r['timings']['검색'],
            '리랭킹': r['timings']['리랭킹'],
            '병합': r['timings']['병합'],
            '컨텍스트': r['timings']['컨텍스트'],
            '답변': r['timings']['답변'],
            '총시간': r['total'],
            '검색수': r['search_count'],
            '병합수': r['merged_count']
        })

df = pd.DataFrame(data)
print("\n" + "=" * 100)
print("RAG 파이프라인 성능 측정 결과")
print("=" * 100)
print(df.to_string(index=False))

# 평균
print("\n" + "=" * 100)
print("평균 소요 시간")
print("=" * 100)
numeric_cols = ['재구성', '멀티쿼리', '검색', '리랭킹', '병합', '컨텍스트', '답변', '총시간']
for col in numeric_cols:
    print(f"{col}: {df[col].mean():.2f}s")

# CSV 저장
csv_file = f"RAG_테스트_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df.to_csv(csv_file, index=False, encoding='utf-8-sig')
print(f"\n결과 저장: {csv_file}")


RAG 파이프라인 성능 측정 결과
   TC                                 질문      재구성     멀티쿼리       검색       리랭킹       병합     컨텍스트       답변       총시간  검색수  병합수
TC-01                    수원시 행복주택 공고 알려줘 1.846615 1.133067 2.028143 13.216126 0.000796 0.000042 9.631698 27.857288   19    9
TC-02            수원매산 A1블록 행복주택 신청자격 알려줘 2.158064 1.996483 1.160675 10.477155 0.000085 0.000030 4.593802 20.386987   17   10
TC-03       수원매산 A1블록 행복주택 우선공급 선정기준 알려줘 1.581852 1.414299 1.011730 11.610797 0.000105 0.000033 3.551303 19.170937   19    9
TC-04 수원매산 A1블록 행복주택 어느 지역 사람이 신청할 수 있어? 1.150282 1.534467 1.194293  8.702135 0.000027 0.000009 2.274101 14.856063   15    7

평균 소요 시간
재구성: 1.68s
멀티쿼리: 1.52s
검색: 1.35s
리랭킹: 11.00s
병합: 0.00s
컨텍스트: 0.00s
답변: 5.01s
총시간: 20.57s

결과 저장: RAG_테스트_20251125_115749.csv


## 5. 테스트 검증 - 4가지 핵심 기능 확인

In [38]:
# ============================================================
# 테스트 검증: 4가지 핵심 기능 확인
# ============================================================

for result in all_results:
    if 'error' in result:
        continue

    print("\n" + "=" * 80)
    print(f"[{result['tc_id']}] {result['query']}")
    print("=" * 80)

    # 1. 질문 재구성 & 멀티쿼리 생성 검증
    print("\n[검증 1] 질문 재구성 및 멀티쿼리 생성")
    print("-" * 60)
    qa = result['query_analysis']
    print(f"원본 질문: {result['query']}")
    print(f"재구성된 질문: {qa.get('rewritten_question')}")
    print(f"추출된 필터:")
    print(f"  - 지역: {qa.get('region') or '없음'}")
    print(f"  - 유형: {qa.get('notice_type') or '없음'}")
    print(f"  - 상태: {qa.get('status') or '없음'}")
    print(f"검색 키워드: {', '.join(qa.get('search_keywords', []))}")
    print(f"\n생성된 멀티쿼리 ({len(result['multi_queries'])}개):")
    for i, mq in enumerate(result['multi_queries'], 1):
        print(f"  {i}. {mq}")

    # 2. 하이브리드 검색 검증
    print(f"\n[검증 2] 하이브리드 검색 - 연관 문서 검색")
    print("-" * 60)
    print(f"총 검색된 문서: {result['search_count']}개")
    print(f"상위 5개 검색 결과:")
    for i, doc in enumerate(result['search_results'][:5], 1):
        title = doc.get('title', 'N/A')[:50]
        region = doc.get('region', 'N/A')
        notice_type = doc.get('notice_type', 'N/A')
        similarity = doc.get('similarity', 0)
        print(f"  [{i}] {title}...")
        print(f"      지역: {region}, 유형: {notice_type}, 유사도: {similarity:.4f}")

    # 3. 리랭킹 & 병합 검증
    print(f"\n[검증 3] 재순위화(Reranking) 및 병합(Merge)")
    print("-" * 60)
    print("리랭킹 전 상위 3개:")
    for i, doc in enumerate(result['search_results'][:3], 1):
        title = doc.get('title', 'N/A')[:40]
        similarity = doc.get('similarity', 0)
        print(f"  [{i}] {title}... (유사도: {similarity:.4f})")

    print("\n리랭킹 후 상위 3개:")
    for i, doc in enumerate(result['reranked'][:3], 1):
        title = doc.get('title', 'N/A')[:40]
        rerank_score = doc.get('rerank_score', 0)
        print(f"  [{i}] {title}... (rerank: {rerank_score:.4f})")

    print(f"\n병합 결과: {result['merged_count']}개 공고로 병합")
    print("병합된 상위 3개 공고:")
    for i, ann in enumerate(result['merged'][:3], 1):
        title = ann.get('announcement_title', 'N/A')
        num_chunks = ann.get('num_chunks', 0)
        content_len = len(ann.get('merged_content', ''))
        print(f"  [{i}] {title}")
        print(f"      병합 청크: {num_chunks}개, 텍스트 길이: {content_len}자")

    # 4. 답변 생성 및 환각 검증
    print(f"\n[검증 4] 답변 생성 - 환각 없는 정확한 답변")
    print("-" * 60)
    answer = result['answer']
    print(f"답변 길이: {len(answer)}자")
    print(f"\n생성된 답변:")
    print("-" * 60)
    print(answer)
    print("-" * 60)

    print("\n")

print("\n" + "=" * 80)
print("전체 테스트 검증 완료")
print("=" * 80)


[TC-01] 수원시 행복주택 공고 알려줘

[검증 1] 질문 재구성 및 멀티쿼리 생성
------------------------------------------------------------
원본 질문: 수원시 행복주택 공고 알려줘
재구성된 질문: 수원시 행복주택 공고 알려줘
추출된 필터:
  - 지역: 없음
  - 유형: 없음
  - 상태: 없음
검색 키워드: 수원시, 행복주택, 공고, 알려줘

생성된 멀티쿼리 (2개):
  1. 수원시 행복주택 공고 알려줘
  2. 수원시 행복주택 공지사항 확인하고 싶어

[검증 2] 하이브리드 검색 - 연관 문서 검색
------------------------------------------------------------
총 검색된 문서: 19개
상위 5개 검색 결과:
  [1] 수원시 지역 행복주택 예비입주자 모집 공고(2025.07.03.)...
      지역: 경기도, 유형: 행복주택, 유사도: 0.6896
  [2] [정정공고]수원시 지역 행복주택 예비입주자 모집 공고(2025.07.03.)...
      지역: 경기도, 유형: 행복주택, 유사도: 0.6896
  [3] [정정공고]의왕초평 A-3블록 신혼희망타운(공공분양) 잔여세대 추가입주자모집공고...
      지역: 경기도, 유형: 공공분양(신혼희망), 유사도: 0.6883
  [4] [정정공고]의왕초평 A-3블록 신혼희망타운(공공분양) 잔여세대 추가입주자모집공고...
      지역: 경기도, 유형: 공공분양(신혼희망), 유사도: 0.6832
  [5] 수원시 지역 행복주택 예비입주자 모집 공고(2025.01.20)...
      지역: 경기도, 유형: 행복주택, 유사도: 0.6792

[검증 3] 재순위화(Reranking) 및 병합(Merge)
------------------------------------------------------------
리랭킹 전 상위 3개:
  [1] 수원시 지역 행복주택 예비입주자 모집 공고(2025.

In [39]:
# TC-04 디버깅: 접수기간 검색 실패 원인 분석
query = "수원매산 A1블록 행복주택 청약 접수기간 알려줘"

# 1. 질문 재구성
query_analysis = await llm_handler.rewrite_query(query, [])
print("=" * 80)
print("1. 질문 재구성")
print("=" * 80)
print(f"검색 키워드: {query_analysis.get('search_keywords')}")

# 2. 멀티쿼리
multi_queries = await llm_handler.generate_multi_queries(query, query_analysis, num_queries=1)
print(f"\n멀티쿼리: {multi_queries}")

# 3. 검색
search_results = await gongo.multi_query_hybrid_search(query_analysis, multi_queries)
print(f"\n=" * 80)
print(f"2. 검색 결과")
print("=" * 80)
print(f"총 {len(search_results)}개 검색됨")

# 수원매산 A1블록(LH_lease_249) 청크만 필터링
target_ann_id = 'LH_lease_249'
suwon_chunks = [r for r in search_results if r.get('announcement_id') == target_ann_id]
print(f"\n수원매산 A1블록 관련 청크: {len(suwon_chunks)}개")
for chunk in suwon_chunks:
    idx = chunk.get('chunk_index')
    sim = chunk.get('similarity', 0)
    text_preview = chunk.get('chunk_text', '')[:150].replace('\n', ' ')
    print(f"  [청크 {idx}] 유사도: {sim:.4f}")
    print(f"  내용: {text_preview}...")
    print()

# 4. 리랭킹
reranked = await gongo.rerank_results(query_analysis.get('rewritten_question', query), search_results, top_k=25)
print("=" * 80)
print("3. 리랭킹 후")
print("=" * 80)
suwon_reranked = [r for r in reranked if r.get('announcement_id') == target_ann_id]
print(f"수원매산 A1블록 관련 청크: {len(suwon_reranked)}개")
for i, chunk in enumerate(suwon_reranked, 1):
    idx = chunk.get('chunk_index')
    score = chunk.get('rerank_score', 0)
    text_preview = chunk.get('chunk_text', '')[:150].replace('\n', ' ')
    print(f"  [{i}] 청크 {idx}, rerank: {score:.4f}")
    print(f"  내용: {text_preview}...")
    print()

# 청크 1, 39번이 포함되었는지 확인
chunk_indices = [c.get('chunk_index') for c in suwon_reranked]
print(f"\n청크 인덱스 목록: {chunk_indices}")
if 1 in chunk_indices:
    print("✅ 청크 1 (접수기간 정보) 포함됨")
else:
    print("❌ 청크 1 (접수기간 정보) 제외됨")
    
if 39 in chunk_indices:
    print("✅ 청크 39 (접수기간 정보) 포함됨")
else:
    print("❌ 청크 39 (접수기간 정보) 제외됨")

1. 질문 재구성
검색 키워드: ['수원매산', 'A1블록', '행복주택', '청약', '접수기간', '알려줘']

멀티쿼리: ['수원매산 A1블록 행복주택 청약 접수기간 알려줘', '수원매산 A1단지 행복주택 신청 기간이 언제인지 알려주세요']

=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
2. 검색 결과
총 21개 검색됨

수원매산 A1블록 관련 청크: 5개
  [청크 27] 유사도: 0.7397
  내용: |Col1|[ 청약신청 주택명 : 수원매산 A1블록 행복주택, 주택관리번호 : 2024-000734 ]| |---|---| |인 터 넷<br>직접 발급|**한국부동산원 청약홈**(www.applyhome.co.kr)**접속** → 화면 중앙의 ‘청약통장 순위확인서’ 선...

  [청크 5] 유사도: 0.6922
  내용: - 수원매산 A1블록 행복주택은 『수원시 매산동 행정복지센터 복합개발사업』에 따라 노후화된 매산동 행정복지  센터 등을 철거하고 그 부지에 신축한 **공공청사 등 복합건물** **[*]** **상층부에 위치한 임대주택** 이며, 금회 모집하는  행복주택의 동·호수 배정...

  [청크 25] 유사도: 0.6222
  내용: |항목|3점|1점| |---|---|---| |① 거주지 및 거주기간|**수원시**에 3년 이상 거주|**수원시**에 3년 미만 거주| |② 주택청약종합저축<br> (청약저축 포함) 납입횟수|가입 2년이 경과한 자로서 매월 약정<br>납입일에 월납입금을 24회 이상 납...

  [청크 31] 유사도: 0.6385
  내용: **▶** **일반공급 경쟁 시 입주자(서류제출대상자 포함) 선정기준 :** **1세 이하의 자녀가 있는 자 → 추첨**  **입주자모집공고일(2024.

## 6. 디버깅: TC-04 접수기간 검색 실패 원인 분석